In [1]:
import os
import time
import warnings

import cv2
import numpy as np
import pandas as pd

from src.generate_patches import CropImage
from src.model_test import AntiSpoofPredict
from src.utility import parse_model_name

warnings.filterwarnings("ignore")

/home/ai/anaconda3/envs/notebook/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
def test(
    image,
    model_dir='resources/ckpt_test/', 
    device_id='0'):
    model_test = AntiSpoofPredict(device_id)
    image_cropper = CropImage()

    image_bbox = model_test.get_bbox(image)
    prediction = np.zeros((1, 2))
    test_speed = 0
    # sum the prediction from single model's result
    for model_name in os.listdir(model_dir):
        h_input, w_input, model_type, scale = parse_model_name(model_name)
        param = {
            "org_img": image,
            "bbox": image_bbox,
            "scale": scale,
            "out_w": w_input,
            "out_h": h_input,
            "crop": True,
        }
        if scale is None:
            param["crop"] = False
        img = image_cropper.crop(**param)
        start = time.time()
        prediction += model_test.predict(img, os.path.join(model_dir, model_name))
        test_speed += time.time() - start

    label = np.argmax(prediction)
    value = prediction[0][label] / 2
    if label == 1:
        return value, test_speed*1000
    else:
        return 1 - value, test_speed*1000

In [3]:
args = {
    'data': 'data/',
    # 'data': "/home/ai/challenge/darf-nam/zalo-challenge/datasets/videos",
    'model_dir':'resources/ckpt/',
    'device_id':0,
}

target = {}
target_test_speed = {}
for idx, video_name in enumerate(os.listdir(args['data'])):
    video_path = os.path.join(args['data'], video_name)
    cap = cv2.VideoCapture(video_path)
    ls = []
    speed = 0
    c = 0
    while cap.isOpened():
        ret, frame = cap.read()
        try:
            if c % 5 == 0:
                score, run_speed = test(frame, args['model_dir'], args['device_id'])
                ls.append(score)
                speed += run_speed

            c += 1
        except:
            break

    target[video_name] = sum(ls) / len(ls)
    target_test_speed[video_name] = speed
    print('{}/{}'.format(idx+1, len(os.listdir(args['data']))), video_name, speed)

df_score = pd.DataFrame(list(target.items()), columns=["fname", "liveness_score"])
df_time = pd.DataFrame(list(target_test_speed.items()), columns=["fname", "time"])

os.makedirs('results/',exist_ok=True)
df_score.to_csv("results/jupyter_submission.csv", index=False, encoding="utf-8", float_format="%.10f")
df_time.to_csv("results/time_submission.csv", index=False, encoding="utf-8", float_format="%.10f")

0/486 452.mp4 4572.906732559204
1/486 458.mp4 2655.6217670440674
2/486 89.mp4 2006.7899227142334
3/486 338.mp4 1382.007360458374
4/486 426.mp4 1841.6697978973389
5/486 156.mp4 2096.8804359436035
6/486 56.mp4 1437.920331954956
7/486 18.mp4 2097.1031188964844
8/486 485.mp4 3767.2603130340576
9/486 339.mp4 2298.8946437835693
10/486 386.mp4 1556.504487991333
11/486 304.mp4 1809.943437576294
12/486 288.mp4 1734.0290546417236
13/486 210.mp4 1800.7757663726807
14/486 421.mp4 1870.384931564331
15/486 325.mp4 3465.9509658813477
16/486 464.mp4 2279.320001602173
17/486 258.mp4 1708.1234455108643
18/486 436.mp4 1931.4067363739014
19/486 232.mp4 1741.1527633666992
20/486 343.mp4 1491.5080070495605
21/486 208.mp4 3476.2320518493652
22/486 4.mp4 1523.400068283081
23/486 364.mp4 1136.2788677215576
24/486 235.mp4 2235.626697540283
25/486 197.mp4 1592.8523540496826
26/486 459.mp4 1282.5639247894287
27/486 133.mp4 3033.2798957824707
28/486 351.mp4 3462.5089168548584
29/486 295.mp4 2173.198461532593
30/48